业务测试：context_providers.py, pipelines.py, tasks.py

context_providers.py测试


In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-02-10 01:58:36,470 storage 使用 COSStorage 初始化存储
INFO 2025-02-10 01:58:36,470 storage 初始化 COSStorage
INFO 2025-02-10 01:58:36,471 storage Storage backend initialization with args: (), kwargs: {}
INFO 2025-02-10 01:58:36,471 storage ✅ 成功初始化 COSStorage
INFO 2025-02-10 01:58:36,472 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
# 准备测试所需的 user, project, file_record  (其中project与file_record关联)
from django.contrib.auth import get_user_model
User = get_user_model()

# 获取已存在的测试数据

# 获取已存在的用户
test_user = User.objects.get(phone='18501771516')
print(f"用户: {test_user.phone}")


用户: 18501771516


In [3]:
# 导入测试模块
from apps.chat.context_providers import ContextManager, UserContextProvider, ConversationContextProvider, RAGContextProvider
from apps.chat.pipelines import ChatPipeline
#from apps.chat.tasks import process_chat_message_task, batch_process_messages_task
from apps.chat.models import ChatSession, ChatMessage
from django.contrib.auth import get_user_model
import uuid
from pprint import pprint


1. 测试上下文管理器

In [ ]:
async def test_context_providers():
    # 创建测试会话
    test_session_id = str(uuid.uuid4())
    
    # 测试单个提供者
    user_provider = UserContextProvider()
    user_context = await user_provider.get_context(test_session_id)
    print("User Context:", user_context)
    
    conv_provider = ConversationContextProvider()
    conv_context = await conv_provider.get_context(test_session_id)
    print("\nConversation Context:", conv_context)
    
    rag_provider = RAGContextProvider()
    rag_context = await rag_provider.get_context(test_session_id)
    print("\nRAG Context:", rag_context)
    
    # 测试组合上下文
    context_manager = ContextManager()
    combined_context = await context_manager.get_combined_context(test_session_id)
    print("\nCombined Context:", combined_context)

    pprint(combined_context)
# 运行测试
await test_context_providers()


2. 测试管道处理 pipeline()

In [5]:
# 创建测试会话和消息
test_session_id = str(uuid.uuid4())
test_message = "你好，请介绍一下你自己。"
 

In [6]:
# 创建管道实例
pipeline = ChatPipeline()

In [ ]:
# 测试单条消息处理
print("Testing single message processing...")
response = await pipeline.process_message(test_session_id, test_message, test_user)
print("Pipeline Response:", response)

In [ ]:
# 测试批量消息处理
print("\nTesting batch message processing...")
test_messages = [
    {"session_id": test_session_id, "content": "第一条测试消息"},
    {"session_id": test_session_id, "content": "第二条测试消息"}
]
batch_responses = await pipeline.process_batch(test_messages,test_user)
print("Batch Responses:", batch_responses)

3. 测试异步任务 tasks.py

In [9]:
# Celery 安装和配置测试 (需要和tasks.py测试代码配套使用)
# 导入任务
from apps.chat.tasks import test_task



In [11]:
from apps.chat.tasks import test_task, process_chat_message_task, batch_process_messages_task
# 打印查看 task 的属性
print(test_task)
print(type(test_task))
print(process_chat_message_task)
print(type(process_chat_message_task))
print(batch_process_messages_task)
print(type(batch_process_messages_task))

# 执行任务
result = test_task.delay()
print(f"test_task_result:{result.get()}")

<@task: chat.test_task of config at 0x18daeaa8dd0>
<class 'celery.local.celery.local'>
<@task: chat.process_chat_message_task of config at 0x18daeaa8dd0>
<class 'celery.local.celery.local'>
<@task: chat.batch_process_messages_task of config at 0x18daeaa8dd0>
<class 'celery.local.celery.local'>
test_task_result:Hello from Celery!


In [ ]:
# 测试单条消息处理任务
print("Testing single message processing task...")
test_session_id = str(uuid.uuid4())

result = process_chat_message_task.delay(session_id = test_session_id, content="这是一条测试消息", user_phone="18501771516")
print("Task ID:", result.id)
print("Task Result:", result.get())

In [ ]:
    # 测试批量消息处理任务
print("\nTesting batch message processing task...")
test_messages = [
    {"session_id": test_session_id, "content": "批量测试消息1"},
    {"session_id": test_session_id, "content": "批量测试消息2"}
]
batch_result = batch_process_messages_task.delay(test_messages, user_phone="18501771516")
print("Batch Task ID:", batch_result.id)
print("Batch Task Result:", batch_result.get())

端到端的测试

In [ ]:
# 创建测试用户和会话  #要用await 需要psycopg3+, 配置，acreate
test_session = await ChatSession.objects.acreate(created_by=test_user)

In [7]:
    # 1. 获取上下文
context_manager = ContextManager()
context = await context_manager.get_combined_context(str(test_session.id))
print("Initial Context:", context)

Initial Context: {'context': {'user_preferences': {'language': 'zh', 'expertise_level': 'intermediate'}, 'session_start_time': '2025-02-09T18:01:00.945801+00:00', 'conversation_metadata': {'total_messages': 0, 'conversation_summary': 'No previous conversation', 'last_interaction_time': None}, 'knowledge_base_info': {'status': 'available', 'last_updated': None}}, 'documents': []}


In [8]:
# 2. 通过管道处理消息
pipeline = ChatPipeline()
response = await pipeline.process_message(
    str(test_session.id),
    "你能帮我解释一下什么是机器学习吗？",
    test_user
)
print("\nPipeline Response:", response)

[Pipelines-Process_message]开始处理会话 daf36a65-6a0e-4b61-8c02-539b2abe5e94 的管道流程
[Pipelines-Process_message]正在执行步骤: PreprocessStep
处理前数据：{'session_id': 'daf36a65-6a0e-4b61-8c02-539b2abe5e94', 'content': '你能帮我解释一下什么是机器学习吗？'}
[Pipelines-PreprocessStep] 开始预处理会话 daf36a65-6a0e-4b61-8c02-539b2abe5e94 的消息
[Pipelines-PreprocessStep]预处理完成，处理后的内容长度: 17
处理后数据：{'session_id': 'daf36a65-6a0e-4b61-8c02-539b2abe5e94', 'content': '你能帮我解释一下什么是机器学习吗？', 'timestamp': '2025-02-09T18:01:50.973247+00:00', 'processed_content': '你能帮我解释一下什么是机器学习吗？'}
[Pipelines-Process_message]正在执行步骤: ContextEnrichmentStep
处理前数据：{'session_id': 'daf36a65-6a0e-4b61-8c02-539b2abe5e94', 'content': '你能帮我解释一下什么是机器学习吗？', 'timestamp': '2025-02-09T18:01:50.973247+00:00', 'processed_content': '你能帮我解释一下什么是机器学习吗？'}
[Pipelines-ContextEnrichmentStep]开始为会话 daf36a65-6a0e-4b61-8c02-539b2abe5e94 进行上下文富集
[Pipelines-ContextEnrichmentStep]上下文富集完成，上下文数据包含以下键: ['context', 'documents']
处理后数据：{'session_id': 'daf36a65-6a0e-4b61-8c02-539b2abe5e94', 'content': 

In [12]:
# 3. 创建异步任务
task_result = process_chat_message_task.delay(
    str(test_session.id),
    "继续解释深度学习的概念",
    user_phone="18501771516"
)
print("\nTask Result:", task_result.get())


Task Result: {'response': '当然，深度学习是机器学习的一个子领域，它通过构建和训练多层的神经网络模型来处理复杂的数据模式。这些模型被称为“深度”是因为它们包含多个隐藏层（通常超过两层），这使得它们能够捕捉数据中的复杂特征和抽象概念。\n\n### 深度学习的核心概念\n\n1. **神经网络**：\n   - **结构**：一个典型的神经网络由输入层、一个或多个隐藏层和输出层组成。每一层都包含若干个节点（或称为神经元），这些节点通过权重连接起来。\n   - **功能**：每个神经元接收来自前一层的输入，对其进行加权求和，并通过一个激活函数进行非线性变换，然后将结果传递给下一层。\n\n2. **深度架构**：\n   - **卷积神经网络（CNN）**：主要用于图像识别任务。CNN通过卷积层提取图像的空间层次特征，如边缘、纹理等，适用于视觉识别、图像分类等领域。\n   - **循环神经网络（RNN）**：用于处理序列数据，如时间序列分析、自然语言处理。RNN具有记忆功能，可以记住之前的信息，从而更好地处理顺序信息。\n   - **长短期记忆网络（LSTM）和门控循环单元（GRU）**：这两种改进型的RNN解决了传统RNN在处理长序列时遇到的梯度消失问题，更擅长捕捉长时间依赖关系。\n   - **Transformer**：近年来兴起的一种架构，广泛应用于自然语言处理任务，如翻译、文本生成等。它摒弃了传统的循环结构，采用自注意力机制来并行处理序列中的所有位置。\n\n3. **训练过程**：\n   - **反向传播算法**：这是深度学习中最常用的优化方法。它通过计算损失函数相对于网络参数的梯度，并使用梯度下降法更新这些参数，以最小化预测误差。\n   - **大数据与高性能计算**：深度学习模型通常需要大量的数据进行训练，并且计算量巨大。因此，GPU（图形处理单元）和TPU（张量处理单元）等硬件加速技术以及分布式计算框架对于提高训练效率至关重要。\n\n4. **应用领域**：\n   - **计算机视觉**：包括图像分类、目标检测、人脸识别等。\n   - **自然语言处理（NLP）**：涉及文本理解、情感分析、机器翻译等。\n   - **语音识别**：将音频信号转换为文本，支持智能助手等功能。\n   - **推荐系统**：

In [20]:
task_result_content = task_result.get()
print(task_result_content['response'])

当然，深度学习是机器学习的一个子领域，它通过构建和训练多层的神经网络模型来处理复杂的数据模式。这些模型被称为“深度”是因为它们包含多个隐藏层（通常超过两层），这使得它们能够捕捉数据中的复杂特征和抽象概念。

### 深度学习的核心概念

1. **神经网络**：
   - **结构**：一个典型的神经网络由输入层、一个或多个隐藏层和输出层组成。每一层都包含若干个节点（或称为神经元），这些节点通过权重连接起来。
   - **功能**：每个神经元接收来自前一层的输入，对其进行加权求和，并通过一个激活函数进行非线性变换，然后将结果传递给下一层。

2. **深度架构**：
   - **卷积神经网络（CNN）**：主要用于图像识别任务。CNN通过卷积层提取图像的空间层次特征，如边缘、纹理等，适用于视觉识别、图像分类等领域。
   - **循环神经网络（RNN）**：用于处理序列数据，如时间序列分析、自然语言处理。RNN具有记忆功能，可以记住之前的信息，从而更好地处理顺序信息。
   - **长短期记忆网络（LSTM）和门控循环单元（GRU）**：这两种改进型的RNN解决了传统RNN在处理长序列时遇到的梯度消失问题，更擅长捕捉长时间依赖关系。
   - **Transformer**：近年来兴起的一种架构，广泛应用于自然语言处理任务，如翻译、文本生成等。它摒弃了传统的循环结构，采用自注意力机制来并行处理序列中的所有位置。

3. **训练过程**：
   - **反向传播算法**：这是深度学习中最常用的优化方法。它通过计算损失函数相对于网络参数的梯度，并使用梯度下降法更新这些参数，以最小化预测误差。
   - **大数据与高性能计算**：深度学习模型通常需要大量的数据进行训练，并且计算量巨大。因此，GPU（图形处理单元）和TPU（张量处理单元）等硬件加速技术以及分布式计算框架对于提高训练效率至关重要。

4. **应用领域**：
   - **计算机视觉**：包括图像分类、目标检测、人脸识别等。
   - **自然语言处理（NLP）**：涉及文本理解、情感分析、机器翻译等。
   - **语音识别**：将音频信号转换为文本，支持智能助手等功能。
   - **推荐系统**：根据用户行为提供个性化推荐，如电影推荐、商品推荐等。
   - **自动驾驶**：利用传感器数据进行